## Exercise 1

In [1]:
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt
import time
import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard 
#from torch.utils.tensorboard import SummaryWriter

2024-04-02 22:44:06.054005: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
training_data = torchvision.datasets.CIFAR10(
    root="data",
    train=True,
    download=True,
    transform=torchvision.transforms.ToTensor()
)

test_data = torchvision.datasets.CIFAR10(
    root="data",
    train=False,
    download=True,
    transform=torchvision.transforms.ToTensor()
)

labels_map = {
    0: "Airplane",
    1: "Automobile",
    2: "Bird",
    3: "Cat",
    4: "Deer",
    5: "Dog",
    6: "Frog",
    7: "Horse",
    8: "Ship",
    9: "Truck",
}

Files already downloaded and verified
Files already downloaded and verified


In [3]:
from torch.utils.data import Dataset

class MyCIFAR10Dataset(Dataset):
    """CIFAR10 dataset."""

    def __init__(self, dataset, classes=torch.tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]), min_max_normalise=1, flatten=0):
        """
        Arguments:
        dataset -- a tuple with the [images, labels] of the original dataset (images should be in a tensor format)
        classes -- list of classes to use for training (at least two classes must be given)
        min_max_normalise -- whether to do min-max-normalisation (1) or rescaling (0)
        flatten -- whether to flatten the 32x32x3 image to a single row (=1);
        """
        self.prepare_data(dataset, classes, min_max_normalise, flatten)

    def __len__(self):
        return self.num_samples

    def __getitem__(self, idx):
        return self.x_sel[idx], self.y_sel[idx]

    def prepare_data(self, dataset, classes, min_max_normalise, flatten):
        x = torch.tensor(dataset[0], dtype=torch.float)
        y = torch.tensor(dataset[1], dtype=torch.long)

        ind_sel = torch.isin(y, classes)
        x_sel = x[ind_sel].clone()
        y_sel = y[ind_sel].clone()

        for i0 in range(len(classes)):
            if i0 != classes[i0]:
                y_sel[y_sel == classes[i0]] = i0

        self.num_samples = x_sel.shape[0]

        if min_max_normalise:
            xmax, xmin = torch.max(x_sel), torch.min(x_sel)
            x_sel = 2 * (x_sel - xmin) / (xmax - xmin) - 1
        else:
            xmax = torch.max(x_sel)
            x_sel = x_sel / xmax

        if flatten:
            m = x_sel.shape[0]
            x_sel = x_sel.view(m, -1)

        self.x_sel = x_sel
        self.y_sel = y_sel

In [4]:
writer = SummaryWriter('tensorboard/cifar10_experiment')

In [5]:
my_images = torch.tensor(training_data.data[:20]).permute(0, 3, 1, 2).float() / 255

img_grid = torchvision.utils.make_grid(my_images.float())

writer.add_image('a_set_of_cifar10_images', img_grid)

In [6]:
class NeuralNetwork:
    """
    MLP class handling the layers and doing all propagation and back propagation steps
    all hidden layers are dense (with ReLU activation) and the last layer is softmax
    """
    def __init__(self, list_num_neurons):
        """
        constructor

        Arguments:
        list_num_neurons -- list of layer sizes including in- and output layer
        
        """
        self.model = torch.nn.Sequential()
        #now we require a flatten tensor
        self.model.add_module('flatten', torch.nn.Flatten(start_dim=1, end_dim=-1))
        #first construct dense layers
        for i0 in range(len(list_num_neurons)-2):
            self.model.add_module('dense' + str(i0), torch.nn.Linear(list_num_neurons[i0], list_num_neurons[i0+1]))
            self.model.add_module('act' + str(i0), torch.nn.ReLU())
            
        #finally add softmax layer
        self.model.add_module('dense' + str(i0+1), torch.nn.Linear(list_num_neurons[-2], list_num_neurons[-1]))
        self.model.add_module('act' + str(i0+1), torch.nn.Softmax(dim=1))
                         
        
        self.cost_fn = torch.nn.CrossEntropyLoss(reduction='mean')
        
        #used to save results
        self.result_data = torch.tensor([])
        
        #we keep a global step counter, thus that optimise can be called 
        #several times with different settings
        self.epoch_counter = 0 
        
    def propagate(self, x):
        """
        calculates the function estimation based on current parameters
        """            
        y_pred = self.model(x)

        return y_pred
           
     
    def back_propagate(self, cost):
        """
        calculates the backpropagation results based on expected output y
        this function must be performed AFTER the corresponding propagte step
        """    
        #set gradient values to zero
        self.model.zero_grad()
              
        cost.backward()
 

    def cost_funct(self, y_pred, y):
        """
        calculates the MSE loss function
        """
        cost = self.cost_fn(y_pred, y)
        
        return cost
    
         
    def gradient_descend(self, alpha):
        """
        does the gradient descend based on results from last back_prop step with learning rate alpha
        """
        with torch.no_grad():
            self.optimizer.step()
            
         
    def calc_error(self, y_pred, y):
        """
        get error information
        """
        m = y.shape[0]

        y_pred_argmax = torch.argmax(y_pred, dim=1)
        error = torch.sum(y != y_pred_argmax) / m

        return error

    
    def append_result(self):
        """
        append cost and error data to output array
        """
        #this takes quite a long time (transform is applied to all images) but is only executed once 
        #then the images are available for quick execution of propagation step
        if self.epoch_counter == 0: 
            # dataloaders (we use original set (training/test_data); own data has to realize the abstract class representing 'Dataset'
            train_loader = torch.utils.data.DataLoader(self.data['train'], batch_size=len(self.data['train']), shuffle=False)
            train_iterator = iter(train_loader)
            self.train_images, self.train_labels = next(train_iterator)
    
            valid_loader = torch.utils.data.DataLoader(self.data['valid'], batch_size=len(self.data['valid']), shuffle=False)
            valid_iterator = iter(valid_loader)
            self.valid_images, self.valid_labels = next(valid_iterator)
      
        # determine cost and error functions for train and validation data
        y_pred_train = self.propagate(self.train_images)
        y_pred_val = self.propagate(self.valid_images)

        res_data = torch.tensor([[self.cost_funct(y_pred_train, self.train_labels), 
                                  self.calc_error(y_pred_train, self.train_labels),
                                  self.cost_funct(y_pred_val, self.valid_labels), 
                                  self.calc_error(y_pred_val, self.valid_labels)]])
        
        self.result_data = torch.cat((self.result_data, res_data), 0)

        #send data to tensorboard   
        writer.add_scalars('loss', {'train': res_data[0, 0].item(), \
                                   'validate': res_data[0, 2].item()}, self.epoch_counter)

        writer.add_scalars('error',{'train': res_data[0, 1].item(), \
                                   'validate': res_data[0, 3].item()}, self.epoch_counter)

        #increase epoch counter here (used for plot routines below)
        self.epoch_counter += 1 
        
        return res_data

        
    def optimise(self, data, epochs, alpha, batch_size=0, debug=0):
        """
        performs epochs number of gradient descend steps and appends result to output array

        Arguments:
        data -- dictionary with NORMALISED data
        epochs -- number of epochs
        alpha -- learning rate
        batch_size -- size of batches (1 = SGD, 1 < .. < n = mini-batch)
        debug -- integer value; get info on gradient descend step every debug-step (0 -> no output)
        """
        #access to data from other methods
        self.data = data

        #we define the optimiser
        self.optimizer = torch.optim.SGD(self.model.parameters(), lr=alpha, momentum=0.)
        #self.optimizer = torch.optim.Adam(self.model.parameters(), lr=alpha)

        # dataloader for training image
        train_loader = torch.utils.data.DataLoader(data['train'], batch_size=batch_size, shuffle=True)
        
        # save results before 1st step
        if self.epoch_counter == 0:
            res_data = self.append_result()

        for i0 in range(0, epochs):    
            #measure time for one epoch
            start=time.time()
            #setup loop over all batchs
            data_iterator = iter(train_loader)
            for batch_iter in data_iterator:
                #do prediction
                y_pred = self.propagate(batch_iter[0])
                #determine the loss 
                cost = self.cost_funct(y_pred, batch_iter[1])
                #determine the error
                self.back_propagate(cost)
                #do the correction step
                self.gradient_descend(alpha)

            #save result
            res_data = self.append_result()

            #end of time measurement
            end=time.time()
            
            if debug and np.mod(i0, debug) == 0:
                print('result after %d epochs (dt=%1.2f s)' % (self.epoch_counter-1, end-start))

        if debug:
            print('result after %d epochs, train: cost %.5f, error %.5f ; validation: cost %.5f, error %.5f'
                  % (self.epoch_counter-1, res_data[0, 0].item(), res_data[0, 1].item(), \
                                                                res_data[0, 2].item(), res_data[0, 3].item()))
                        
            

In [13]:
# Choose the categories
classes = torch.tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

# Split data into training and validation sets
validation_size = 0.2
valid_ind = int(len(training_data) * (1 - validation_size))

# Create custom training and validation datasets
train_dataset = MyCIFAR10Dataset([training_data.data[:valid_ind], torch.tensor(training_data.targets)[:valid_ind]], classes=classes)
valid_dataset = MyCIFAR10Dataset([training_data.data[valid_ind:], torch.tensor(training_data.targets)[valid_ind:]], classes=classes)

# Data is arranged as a dictionary with quick access through respective keys
data = {'train': train_dataset, 'valid': valid_dataset}

# Choose the hyperparameters you want to use for the initialisation
# Since CIFAR10 images are 32x32 with 3 channels, adjust size_in accordingly
size_in = 32 * 32 * 3 # For flattened CIFAR10 images
size_out = 10

print('size_in: %d, size_out: %d' % (size_in, size_out))
list_num_neurons = [size_in, 100, size_out]
NNet = NeuralNetwork(list_num_neurons)

/var/folders/d8/m1_4xgr97g573bqjryrsl9yc0000gn/T/ipykernel_8343/3399414968.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.tensor(dataset[1], dtype=torch.long)


size_in: 3072, size_out: 10


In [14]:
writer.add_graph(NNet.model, my_images.float())
writer.close()

In [15]:
epochs = 20
batchsize = 16
learning_rate = 0.05
NNet.optimise(data, epochs, learning_rate, batchsize, debug=5)

result after 1 epochs (dt=2.39 s)
result after 6 epochs (dt=2.21 s)
result after 11 epochs (dt=2.27 s)
result after 16 epochs (dt=2.20 s)
result after 20 epochs, train: cost 1.84541, error 0.38035 ; validation: cost 1.96771, error 0.51020


In [16]:
test_dataset = MyCIFAR10Dataset([test_data.data, test_data.targets], classes=classes)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=len(test_dataset), shuffle=False)
test_iterator = iter(test_loader)
test_images, test_labels = next(test_iterator)

y_pred = torch.argmax(NNet.propagate(test_images), axis=1)
false_classifications = test_images[(y_pred != test_labels)]

print('test error rate: %.2f %% out of %d' % (100*false_classifications.shape[0]/y_pred.shape[0], y_pred.shape[0]))
print(false_classifications.shape)

test error rate: 50.47 % out of 10000
torch.Size([5047, 32, 32, 3])


## Exercise 2

In [3]:
x = training_data.data
x = np.append(x, test_data.data,0)

y = training_data.targets
y = np.append(y, test_data.targets)

print(x.shape)
print(y.shape)

(60000, 32, 32, 3)
(60000,)


In [4]:
def prepare_cifar10_data(x, y, classes, train_size=0.8, min_max_normalise=1, flatten=1):
    """
    Prepares CIFAR10 data for training and testing.
    
    Parameters:
    - x: Images in the dataset.
    - y: Labels corresponding to the images.
    - classes: List of class indices to use.
    - train_size: Proportion of the dataset to use for training.
    - min_max_normalise: Whether to apply min-max normalization.
    - flatten: Whether to flatten the images to a single vector.
    
    Returns:
    - x_train: Training images.
    - x_test: Test images.
    - y_train: Training labels in one-hot format.
    - y_test: Test labels in one-hot format.
    """

    # Select specified classes
    ind_sel = np.isin(y, classes)
    x_sel = x[ind_sel].copy()
    y_sel = y[ind_sel].copy()

    # Replace the labels to be in successive order
    for i, class_id in enumerate(classes):
        y_sel[y_sel == class_id] = i

    # Split data into training and testing
    num_samples = x_sel.shape[0]
    max_train_ind = int(train_size * num_samples)
    indices = np.arange(num_samples)
    np.random.shuffle(indices)
    
    x_train, x_test = x_sel[indices[:max_train_ind]], x_sel[indices[max_train_ind:]]
    y_train, y_test = y_sel[indices[:max_train_ind]], y_sel[indices[max_train_ind:]]

    # Normalize data
    if min_max_normalise:
        xmax, xmin = np.max(x_train), np.min(x_train)
        x_train = 2 * (x_train.astype(float) - xmin) / (xmax - xmin) - 1
        x_test = 2 * (x_test.astype(float) - xmin) / (xmax - xmin) - 1
    else:
        xmax = np.max(x_train)
        x_train = x_train.astype(float) / xmax
        x_test = x_test.astype(float) / xmax

    # Flatten images if specified
    if flatten:
        x_train = x_train.reshape(x_train.shape[0], -1)
        x_test = x_test.reshape(x_test.shape[0], -1)

    # Convert labels to one-hot encoding
    y_train_one_hot = tf.keras.utils.to_categorical(y_train, num_classes=len(classes))
    y_test_one_hot = tf.keras.utils.to_categorical(y_test, num_classes=len(classes))

    return x_train, x_test, y_train_one_hot, y_test_one_hot

In [5]:
class NeuralNetworkCIFAR10:
    """
    MLP class for CIFAR10 dataset. Handles layers, forward propagation, and backpropagation.
    Uses ReLU activations for hidden layers and softmax for the output layer.
    """
    def __init__(self, list_num_neurons, alpha):
        """
        Constructor

        Arguments:
        list_num_neurons -- list of layer sizes including input and output layers
        alpha -- learning rate
        """
        self.model = tf.keras.Sequential()
        
        # Adjust the input shape for CIFAR10 images (32x32 pixels with 3 color channels)
        self.model.add(tf.keras.layers.Flatten(input_shape=(32, 32, 3)))
        
        # Add Dense layers with ReLU activation, except for the output layer which uses softmax
        for num_neurons in list_num_neurons[:-1]:
            self.model.add(tf.keras.layers.Dense(num_neurons, activation='relu'))
        
        # Add the softmax layer for the output
        self.model.add(tf.keras.layers.Dense(list_num_neurons[-1], activation='softmax'))

        print(self.model.summary())
                         
        # Choose the optimizer
        optimizer = tf.keras.optimizers.SGD(learning_rate=alpha)
        
        # Compile the model
        self.model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
         
    def optimise(self, x_train, y_train, epochs, valid_size=0.2, batch_size=16, debug=0, call_backs=None):
        """
        Trains the model on the provided data.

        Arguments:
        x_train -- training data images
        y_train -- training data labels
        epochs -- number of epochs to train for
        valid_size -- fraction of the training data to use as validation
        batch_size -- batch size for training
        debug -- verbosity mode
        call_backs -- list of callbacks for training
        """
        
        # If no callbacks are provided, initialize with an empty list
        if call_backs is None:
            call_backs = []

        # Create a TensorBoard callback
        tensorboard_callback = TensorBoard(log_dir='./logs', histogram_freq=1)

        # Add the TensorBoard callback to the list of callbacks
        call_backs.append(tensorboard_callback)
         
        # Start training
        self.history = self.model.fit(x_train, y_train, validation_split=valid_size,   
                                      batch_size=batch_size, epochs=epochs,
                                      callbacks=call_backs, verbose=debug)

In [7]:
#choose the categories
classes = [0,1,2,3,4,5,6,7,8,9]

#y_train is of type onehot (y_test not!)
x_train, x_test, y_train, y_test = prepare_cifar10_data(x, y, classes, train_size=0.8, min_max_normalise=1, flatten=0)

#choose the hyperparameters you want to use for the initialisation
size_out = len(classes)
list_num_neurons = [100, size_out]; 
learning_rate = 0.05
NNet = NeuralNetworkCIFAR10(list_num_neurons, learning_rate)

#choose the hyperparameters you want to use for training
epochs = 20
batchsize = 16
NNet.optimise(x_train, y_train, epochs, valid_size=0.2, batch_size=batchsize, debug=2)

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten_1 (Flatten)             │ (None, 3072)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 100)            │       307,300 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 10)             │         1,010 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 308,310 (1.18 MB)

 Trainable params: 308,310 (1.18 MB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/20
2400/2400 - 5s - 2ms/step - accuracy: 0.3987 - loss: 1.7178 - val_accuracy: 0.4473 - val_loss: 1.5882
Epoch 2/20
2400/2400 - 5s - 2ms/step - accuracy: 0.4599 - loss: 1.5504 - val_accuracy: 0.4691 - val_loss: 1.5395
Epoch 3/20
2400/2400 - 4s - 2ms/step - accuracy: 0.4883 - loss: 1.4721 - val_accuracy: 0.4590 - val_loss: 1.5694
Epoch 4/20
2400/2400 - 4s - 2ms/step - accuracy: 0.5062 - loss: 1.4234 - val_accuracy: 0.4663 - val_loss: 1.5828
Epoch 5/20
2400/2400 - 4s - 2ms/step - accuracy: 0.5216 - loss: 1.3832 - val_accuracy: 0.4652 - val_loss: 1.5999
Epoch 6/20
2400/2400 - 4s - 1ms/step - accuracy: 0.5354 - loss: 1.3466 - val_accuracy: 0.4795 - val_loss: 1.5582
Epoch 7/20
2400/2400 - 4s - 1ms/step - accuracy: 0.5491 - loss: 1.3065 - val_accuracy: 0.4795 - val_loss: 1.5882
Epoch 8/20
2400/2400 - 4s - 1ms/step - accuracy: 0.5588 - loss: 1.2808 - val_accuracy: 0.4706 - val_loss: 1.6334
Epoch 9/20
2400/2400 - 4s - 1ms/step - accuracy: 0.5732 - loss: 1.2424 - val_accuracy: 0.46